In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

import time

from math import sqrt
np.random.seed(1)

In [3]:
ball = pd.read_csv('Half Ball Prediction Data.csv')
convex = pd.read_csv('Convex Prediction Data.csv')
freeform = pd.read_csv('Freeform 2 Prediction Data.csv')

test_data = pd.read_csv("Whole Test Grid.csv")

test_total = []
for i in range(1,51):
    nameX = "Scan" + str(i) + "X"
    nameY = "Scan" + str(i) + "Y"
    nameZ = "Scan" + str(i) + "Z"
    nameCurv = "Scan" + str(i) + "curvature"
    nameAngle = "Scan" + str(i) + "angle"
    scan_i = test_data[[nameX, nameY, nameZ,nameCurv,nameAngle]]
    scan_i.columns = ['X', 'Y', 'Z','curvature','angle']
    test_total.append(scan_i)

test_d = test_total[2]
    
true_variance = pd.read_csv('True Variance.csv')

In [4]:
def hyperbolic(x):
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

def generate_neurons_uniform(X,num_neurons):
    k = X.shape[1]
    variances = np.std(X,axis=0)
    weights = []
    for i in range(k):
        λ = variances[i]/np.sum(variances)
        a = 2.5 * λ / np.max(np.abs(X[:,i]))
        weight = np.random.uniform(low=-a,high=a,size=num_neurons)
        weights.append(weight)
    weights = np.array(weights)
    intercept = np.random.uniform(low=-1,high=1,size=num_neurons)
    
    return weights, intercept

def cal_neuron_val(X, weights, intercept):
    neuron_val = np.matmul(X,weights) + intercept
    neuron_val = hyperbolic(neuron_val)
    return neuron_val

In [5]:
# BELM
def mbelm(data,coordinate,num_neurons=50):
    weights, intercept = generate_neurons_uniform(data[['Mean Curvature','Mean Angle']].values,num_neurons=num_neurons)
    neuron_val = cal_neuron_val(data[['Mean Curvature','Mean Angle']].values,weights,intercept)
    clf = BayesianRidge()
    name = 'Pointwise Variance ' + coordinate
    clf.fit(neuron_val, data[name].values)
    return {'weight':weights,'intercept':intercept,'model':clf}

def mbelm_pred(train,model):
    neuron_pred = cal_neuron_val(train[['curvature','angle']].values, model['weight'],model['intercept'])
    return model['model'].predict(neuron_pred)

In [6]:
# E-BELM
def mbelm_ensemble(data,coordinate,num_neurons=3, ensemble_num = 50):
    ensemble_weights = []
    ensemble_intercept = []
    ensemble_model = []
    
    for i in range(ensemble_num):
    
        weights, intercept = generate_neurons_uniform(data[['Mean Curvature','Mean Angle']].values,
                                                      num_neurons=num_neurons)
        neuron_val = cal_neuron_val(data[['Mean Curvature','Mean Angle']].values,weights,intercept)

        # Fit the output layer
        clf = BayesianRidge()
        name = 'Pointwise Variance ' + coordinate
        clf.fit(neuron_val, data[name].values)
        
        ensemble_weights.append(weights)
        ensemble_intercept.append(intercept)
        ensemble_model.append(clf)
    
    return {'weight':ensemble_weights,'intercept':ensemble_intercept, 
            'model':ensemble_model, "num": ensemble_num}

def mbelm_pred_ensemble(train,model):
    prediction = 0
    for i in range(model['num']):
        neuron_pred = cal_neuron_val(train[['curvature','angle']].values, 
                                     model['weight'][i],
                                     model['intercept'][i])
        prediction += model['model'][i].predict(neuron_pred)
        
    prediction = prediction/model['num']
    
    return prediction

In [7]:
# Res-BELM
def residual_mbelm(data,coordinate,num_neurons=40, num_residual_nets=5):
    nets_weights = []
    nets_intercept = []
    nets_model = []
    
    name = 'Pointwise Variance ' + coordinate
    
    residual = data[name].values
    
    for i in range(num_residual_nets):
    
        weights, intercept = generate_neurons_uniform(data[['Mean Curvature','Mean Angle']].values,
                                                      num_neurons=num_neurons)
        neuron_val = cal_neuron_val(data[['Mean Curvature','Mean Angle']].values,weights,intercept)

        # Fit the output layer
        clf = BayesianRidge()
        
        clf.fit(neuron_val, residual)
        
        residual = residual - clf.predict(neuron_val)
        
        nets_weights.append(weights)
        nets_intercept.append(intercept)
        nets_model.append(clf)
    
    return {'weight':nets_weights,'intercept': nets_intercept, 
            'model':nets_model, "num": num_residual_nets}

def residual_mbelm_pred(train,model):
    prediction = 0
    for i in range(model['num']):
        neuron_pred = cal_neuron_val(train[['curvature','angle']].values, 
                                     model['weight'][i],
                                     model['intercept'][i])
        
        prediction += model['model'][i].predict(neuron_pred)
        
    prediction = prediction
    
    return prediction

In [8]:
# Training sets
ball = pd.read_csv('Half Ball Prediction Data.csv')
convex = pd.read_csv('Convex Prediction Data.csv')
freeform = pd.read_csv('Freeform 2 Prediction Data.csv')

ball['Pointwise Variance X'] = np.log(ball['Pointwise Variance X'])
ball['Pointwise Variance Y'] = np.log(ball['Pointwise Variance Y'])
ball['Pointwise Variance Z'] = np.log(ball['Pointwise Variance Z'])

convex['Pointwise Variance X'] = np.log(convex['Pointwise Variance X'])
convex['Pointwise Variance Y'] = np.log(convex['Pointwise Variance Y'])
convex['Pointwise Variance Z'] = np.log(convex['Pointwise Variance Z'])

freeform['Pointwise Variance X'] = np.log(freeform['Pointwise Variance X'])
freeform['Pointwise Variance Y'] = np.log(freeform['Pointwise Variance Y'])
freeform['Pointwise Variance Z'] = np.log(freeform['Pointwise Variance Z'])

In [9]:
ball_downsample = ball.sample(n=10000)
convex_downsample = convex.sample(n=10000)
freeform_downsample = freeform.sample(n=10000)

In [10]:
def benchmarking(sample, coordinate='X', test_d=test_d, test_target=true_variance):
    
    mlp = MLPRegressor(random_state=0, max_iter=1000)
    gpr = GaussianProcessRegressor(n_restarts_optimizer=9, alpha=0.1)
    rfr = RandomForestRegressor(random_state=0)
    svr = SVR()

    models = [mlp, gpr, rfr, svr]
    model_name = ['Neural Network', 'Gaussian Process', 'Random Forests', 'Support Vector Machine']
    
    feature = sample[['Mean Curvature','Mean Angle']].values
    target_name = 'Pointwise Variance ' + coordinate 
    target = sample[target_name]
    
    test_feature = test_d[['curvature', 'angle']].values
    
    test_target_name = coordinate + ' Variance'
    test_var = test_target[test_target_name]
    
    output_metrics = pd.DataFrame()
    for i in range(len(models)):
        train_start = time.time()
        models[i].fit(feature, target)
        train_end = time.time()
        training_time = train_end - train_start

        test_start = time.time()
        pred = models[i].predict(test_feature)
        test_end = time.time()
        test_time = test_end - test_start

        rmse = sqrt(mean_squared_error(test_var, np.exp(pred)))

        output_metrics = output_metrics.append(pd.DataFrame({"Model": [model_name[i]], "Training Time": [training_time],
                                                             "Test Time": [test_time], "RMSE": [rmse]}))

    train_start = time.time()
    belm_x = mbelm(sample,coordinate)
    train_end = time.time()
    training_time = train_end - train_start

    test_start = time.time()
    pred = mbelm_pred(test_d, belm_x)
    test_end = time.time()
    test_time = test_end - test_start

    rmse = sqrt(mean_squared_error(test_var, np.exp(pred)))
    output_metrics = output_metrics.append(pd.DataFrame({"Model": ["Single BELM"], "Training Time": [training_time],
                                                         "Test Time": [test_time], "RMSE": [rmse]}))

    train_start = time.time()
    e_belm_x = mbelm_ensemble(sample,coordinate)
    train_end = time.time()
    training_time = train_end - train_start

    test_start = time.time()
    pred = mbelm_pred_ensemble(test_d, e_belm_x)
    test_end = time.time()
    test_time = test_end - test_start

    rmse = sqrt(mean_squared_error(test_var, np.exp(pred)))
    output_metrics = output_metrics.append(pd.DataFrame({"Model": ["E-BELM"], "Training Time": [training_time],
                                                         "Test Time": [test_time], "RMSE": [rmse]}))

    train_start = time.time()
    res_belm_x = residual_mbelm(sample,coordinate)
    train_end = time.time()
    training_time = train_end - train_start

    test_start = time.time()
    pred = residual_mbelm_pred(test_d, res_belm_x)
    test_end = time.time()
    test_time = test_end - test_start

    rmse = sqrt(mean_squared_error(test_var, np.exp(pred)))
    output_metrics = output_metrics.append(pd.DataFrame({"Model": ["Res-BELM"], "Training Time": [training_time],
                                                         "Test Time": [test_time], "RMSE": [rmse]}))
    
    return output_metrics

In [11]:
# Ball x-coordinate
benchmarking(sample=ball_downsample, coordinate='X', test_d=test_d, test_target=true_variance)

,Model,Training Time,Test Time,RMSE
0,Neural Network,1.250067,0.008001,0.003445
0,Gaussian Process,6.953313,4.094848,0.006608
0,Random Forests,1.772200,0.208981,0.002050
0,Support Vector Machine,4.013625,21.239626,0.001264
0,Single BELM,0.046455,0.054003,0.002252
0,E-BELM,0.188923,0.110540,0.002635
0,Res-BELM,0.252887,0.213348,0.001354


In [12]:
# Ball y-coordinate
benchmarking(sample=ball_downsample, coordinate='Y', test_d=test_d, test_target=true_variance)

,Model,Training Time,Test Time,RMSE
0,Neural Network,1.719038,0.007938,0.002812
0,Gaussian Process,8.673758,4.192300,0.003115
0,Random Forests,1.880968,0.199715,0.003265
0,Support Vector Machine,4.080175,19.385508,0.003368
0,Single BELM,0.035799,0.058268,0.003134
0,E-BELM,0.136852,0.095290,0.003142
0,Res-BELM,0.213451,0.190110,0.003491


In [13]:
# Ball z-coordinate
benchmarking(sample=ball_downsample, coordinate='Z', test_d=test_d, test_target=true_variance)

,Model,Training Time,Test Time,RMSE
0,Neural Network,3.749953,0.009778,0.048872
0,Gaussian Process,8.880826,4.195550,0.023006
0,Random Forests,2.009686,0.216599,0.059980
0,Support Vector Machine,3.679471,19.157609,0.030426
0,Single BELM,0.034167,0.054569,0.550975
0,E-BELM,0.180469,0.115930,0.031282
0,Res-BELM,0.242268,0.208769,0.029457


In [14]:
# Freeform 1 x-coordinate
benchmarking(sample=convex_downsample, coordinate='X', test_d=test_d, test_target=true_variance)

,Model,Training Time,Test Time,RMSE
0,Neural Network,5.901029,0.081997,0.001522
0,Gaussian Process,8.841603,4.124676,0.001531
0,Random Forests,2.024151,0.419449,0.001718
0,Support Vector Machine,4.286609,20.318139,0.001549
0,Single BELM,0.031134,0.050252,0.002492
0,E-BELM,0.217005,0.124910,0.001424
0,Res-BELM,0.227311,0.224406,0.002107


In [15]:
# Freeform 1 y-coordinate
benchmarking(sample=convex_downsample, coordinate='Y', test_d=test_d, test_target=true_variance)

,Model,Training Time,Test Time,RMSE
0,Neural Network,1.312164,0.030066,0.003365
0,Gaussian Process,8.730358,4.095485,0.003480
0,Random Forests,1.915028,0.419071,0.003437
0,Support Vector Machine,4.233073,20.671688,0.003315
0,Single BELM,0.030647,0.053005,0.003495
0,E-BELM,0.144804,0.090622,0.003445
0,Res-BELM,0.170620,0.225061,0.003499


In [16]:
# Freeform 1 z-coordinate
benchmarking(sample=convex_downsample, coordinate='Z', test_d=test_d, test_target=true_variance)

,Model,Training Time,Test Time,RMSE
0,Neural Network,1.317853,0.008001,0.031872
0,Gaussian Process,8.792222,4.099079,0.031142
0,Random Forests,1.843828,0.349668,0.030886
0,Support Vector Machine,4.192191,18.807452,0.031006
0,Single BELM,0.034795,0.066197,0.029593
0,E-BELM,0.142767,0.193529,0.031826
0,Res-BELM,0.220596,0.228356,0.029066


In [17]:
# Freeform 2 x-coordinate
benchmarking(sample=freeform_downsample, coordinate='X', test_d=test_d, test_target=true_variance)

,Model,Training Time,Test Time,RMSE
0,Neural Network,2.471773,0.008004,0.001677
0,Gaussian Process,8.760079,4.166181,0.001677
0,Random Forests,1.808593,0.444504,0.001672
0,Support Vector Machine,4.073886,20.837132,0.001666
0,Single BELM,0.030264,0.058028,0.001677
0,E-BELM,0.208188,0.170681,0.001678
0,Res-BELM,0.278574,0.188249,0.001676


In [18]:
# Freeform 2 y-coordinate
benchmarking(sample=freeform_downsample, coordinate='Y', test_d=test_d, test_target=true_variance)

,Model,Training Time,Test Time,RMSE
0,Neural Network,3.410848,0.008001,0.003035
0,Gaussian Process,8.768404,4.045801,0.003260
0,Random Forests,1.773548,0.452702,0.002864
0,Support Vector Machine,3.808194,19.720384,0.002514
0,Single BELM,0.028189,0.065005,0.003223
0,E-BELM,0.209897,0.140529,0.003264
0,Res-BELM,0.211666,0.183881,0.003285


In [19]:
# Freeform 2 z-coordinate
benchmarking(sample=freeform_downsample, coordinate='Z', test_d=test_d, test_target=true_variance)

,Model,Training Time,Test Time,RMSE
0,Neural Network,3.320409,0.007588,0.029331
0,Gaussian Process,8.738807,4.160612,0.028279
0,Random Forests,1.666861,0.434838,0.027967
0,Support Vector Machine,4.045697,18.613415,0.029125
0,Single BELM,0.036539,0.054005,0.029290
0,E-BELM,0.172632,0.146250,0.030123
0,Res-BELM,0.247595,0.187443,0.029147
